![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/30.Clinical_Text_Classification_with_Spark_NLP.ipynb)

# Spark Setup

In [1]:
from johnsnowlabs import nlp, medical
spark = nlp.start()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8,application_1693496626641_0009,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Spark Session already created, some configs may not take.
📋 Loading license number 0 from /lib/.johnsnowlabs/johnsnowlabs/licenses/license_number_0_for_Spark-Healthcare.json

In [2]:
# !pip install -q tensorflow==2.11.0
# !pip install -q tensorflow-addons

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.base import *
from sparknlp_jsl.annotator import *

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline,PipelineModel

import warnings
warnings.filterwarnings('ignore')

# params = {"spark.driver.memory":"24G",
#           "spark.kryoserializer.buffer.max":"2000M",
#           "spark.driver.maxResultSize":"2000M"}

# spark = sparknlp_jsl.start(license_keys['SECRET'],params=params, gpu=True)

# print ("Spark NLP Version :", sparknlp.version())
# print ("Spark NLP_JSL Version :", sparknlp_jsl.version())

# spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Classifiers

The below classifiers will be used in this notebook.ClassifierDL, MultiClassifierDL, and GenericClassifier will be trained using healthcare_100d, embeddingd_clinical, and bert sentence embeddings(sbiobert_base_cased_mli). DocumentLogRegClassifier accepts tokens, so sentence embeddings are not utilized during DocumentLogRegClassifier training.

## ClassifierDL

ClassifierDL is a generic Multi-class Text Classification annotator. ClassifierDL uses the state-of-the-art Universal Sentence Encoder as an input for text classifications. The ClassifierDL annotator uses a deep learning model (DNNs) built inside TensorFlow and supports up to 100 classes. For more information please [follow the link](https://nlp.johnsnowlabs.com/docs/en/annotators#classifierdl).


##  MultiClassifierDL

MultiClassifierDL is a Multi-label Text Classification annotator.MultiClassifierDL uses a Bidirectional GRU with a convolutional model built inside TensorFlow and supports up to 100 classes.  For more information please [follow the link](https://nlp.johnsnowlabs.com/docs/en/annotators#multiclassifierdl).


## GenericClassifier

GenericClassifier is a TensorFlow model for the generic classification of feature vectors in Healthcare  Lİbrary. It takes FEATURE_VECTOR annotations from FeaturesAssembler as input, classifies them, and outputs CATEGORY annotations. Please see [the link](https://nlp.johnsnowlabs.com/docs/en/licensed_annotators#genericclassifier) for more information.

Here are some Social Determinants of Health (SDOH) models that trained with GenericClassifier:

|index|model|
|-----:|:-----|
|1|[genericclassifier_sdoh_economics_binary_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/01/14/genericclassifier_sdoh_economics_binary_sbiobert_cased_mli_en.html)|
|2|[genericclassifier_sdoh_substance_usage_binary_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/01/14/genericclassifier_sdoh_substance_usage_binary_sbiobert_cased_mli_en.html)|
|3|[genericclassifier_sdoh_tobacco_usage_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/01/14/genericclassifier_sdoh_tobacco_usage_sbiobert_cased_mli_en.html)|
|4|[genericclassifier_sdoh_alcohol_usage_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/01/14/genericclassifier_sdoh_alcohol_usage_sbiobert_cased_mli_en.html)|
|5|[genericclassifier_sdoh_alcohol_usage_binary_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/01/14/genericclassifier_sdoh_alcohol_usage_binary_sbiobert_cased_mli_en.html)|
|6|[genericclassifier_sdoh_housing_insecurity_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/04/10/genericclassifier_sdoh_housing_insecurity_sbiobert_cased_mli_en.html)
|7|[genericclassifier_sdoh_mental_health_clinical](https://nlp.johnsnowlabs.com/2023/04/10/genericclassifier_sdoh_mental_health_clinical_en.html)
|8|[genericclassifier_sdoh_under_treatment_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/04/10/genericclassifier_sdoh_under_treatment_sbiobert_cased_mli_en.html)

## GenericLogRegClassifier

`GenericLogRegClassifier` is a derivative of GenericClassifier which implements a multinomial *Logistic Regression*. This is a single layer neural network with the logistic function at the output. The input to the model is `FeatureVector` and the output is `Category` annotations with labels and corresponding confidence scores varying between 0 and 1. Training data requires "text" and their "label" columns only and the trained model will be a `GenericLogRegClassifierModel()`.

|index|model|
|-----:|:-----|
|1|[generic_logreg_classifier_ade](https://nlp.johnsnowlabs.com/2023/05/09/generic_logreg_classifier_ade_en.html)



## GenericSVMClassifier

`GenericSVMClassifier` is a derivative of GenericClassifier which implements *SVM (Support Vector Machine)* classification. The input to the model is `FeatureVector` and the output is `Category` annotations with labels and corresponding confidence scores. The scores are standardized using the logistic function so that they vary between 0 and 1. Taining data requires "text" and their "label" columns only and the trained model will be a `GenericSVMClassifierModel()`

|index|model|
|-----:|:-----|
|1|[generic_svm_classifier_ade](https://nlp.johnsnowlabs.com/2023/05/09/generic_svm_classifier_ade_en.html)

## DocumentLogRegClassifier

DocumentLogRegClassifier is a model to classify documents with a Logarithmic Regression algorithm in Healthcare  Library. Training data requires columns for text and labels. The result is a trained DocumentLogRegClassifierModel. you can get more info [here](https://nlp.johnsnowlabs.com/docs/en/licensed_annotators#documentlogregclassifier).

|index|model|
|-----:|:-----|
|1|[classifier_logreg_ade](https://nlp.johnsnowlabs.com/2023/05/16/classifier_logreg_ade_en.html)|


# ADE Dataset

### Data Preprocessing

In [5]:
#downloading sample datasets

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
%%sh
wget -q -O /tmp/DRUG-AE.rel https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/ADE_Corpus_V2/DRUG-AE.rel

In [4]:
# %%sh
# hdfs dfs -copyFromLocal /tmp/DRUG-AE.rel /user/hadoop/DRUG-AE.rel

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
%%sh
wget -q -O /tmp/ADE-NEG.txt https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/ADE_Corpus_V2/ADE-NEG.txt

In [6]:
# %%sh
# hdfs dfs -copyFromLocal /tmp/ADE-NEG.tx /user/hadoop/ADE-NEG.tx

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**ADE Negative Dataset**

In [7]:
df_neg= pd.read_csv("/tmp/ADE-NEG.txt", header=None, delimiter="\t", names=["col1"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
%%display
spark.createDataFrame(df_neg.head())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [9]:
df_neg['text'] =  df_neg.col1.str.split('NEG').str[1]
df_neg["category"] = "neg"
df_neg= df_neg[["text", "category"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
%%display
spark.createDataFrame(df_neg.head())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

**ADE Positive Dataset**

In [11]:
df_pos= pd.read_csv("/tmp/DRUG-AE.rel", header=None, delimiter="|")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
%%display
spark.createDataFrame(df_pos.head())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [13]:
df_pos["category"]= "pos"
df_pos.rename(columns={1: "text"}, inplace=True)
df_pos= df_pos[["text", "category"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
%%display
spark.createDataFrame(df_pos.head())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

**Merging Positive and Negative dataset**

In [15]:
ade_df= pd.concat([df_neg, df_pos])
# ade_df.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
%%display
spark.createDataFrame(ade_df.head())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [17]:
ade_df["category"].value_counts()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

neg    16695
pos     6821
Name: category, dtype: int64

In [18]:
ade_df.info()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23516 entries, 0 to 6820
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      23516 non-null  object
 1   category  23516 non-null  object
dtypes: object(2)
memory usage: 551.2+ KB

We take 30% of the data to make a faster run. You can use all data for better scores.

In [19]:
spark_df = spark.createDataFrame(ade_df).sample(0.3, 3) # limit the data

trainingData, testData = spark_df.randomSplit([0.8, 0.2], seed = 100)

print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Training Dataset Count: 5531
Test Dataset Count: 1427

In [20]:
trainingData.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------+
|                text|category|
+--------------------+--------+
| "Syndrome malin"...|     neg|
| 'Bail-out' bival...|     neg|
| (4) The acute re...|     neg|
| 2 months later s...|     neg|
| 2-Chlordeoxyaden...|     neg|
| 2. Prior to the ...|     neg|
| 4-en-VPA (2-prop...|     neg|
| 4. More than hal...|     neg|
| A 13-year-old bo...|     neg|
| A 16-year-old gi...|     neg|
| A 16-year-old gi...|     neg|
| A 21-year-old wo...|     neg|
| A 27-year-old ma...|     neg|
| A 28 year-old ma...|     neg|
| A 28-y-old male ...|     neg|
| A 28-year-old fe...|     neg|
| A 3-year-old boy...|     neg|
| A 31-year-old Bl...|     neg|
| A 31-year-old, w...|     neg|
| A 32-year-old ma...|     neg|
+--------------------+--------+
only showing top 20 rows

In [21]:
spark_df.groupBy("category").count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----+
|category|count|
+--------+-----+
|     pos| 2026|
|     neg| 4932|
+--------+-----+

In [22]:
spark_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)

In [23]:
spark_df.head(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(text=' Clioquinol intoxication occurring in the treatment of acrodermatitis enteropathica with reference to SMON outside of Japan.', category='neg'), Row(text=' A 42-year-old woman had uneventful bilateral laser-assisted subepithelial keratectomy (LASEK) to correct myopia.', category='neg'), Row(text=' A 16-year-old girl with erosive, polyarticular JRA showed no detectable change in her articular disease following nine exchanges.', category='neg')]

## 100 Dimension Healthcare Embeddings (embeddings_healthcare_100d)



Now we will extract [healthcare_100d embeddings](https://nlp.johnsnowlabs.com/2020/05/29/embeddings_healthcare_100d_en.html) and use it in the classificaiton model training.

In [24]:
document_assembler = DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

tokenizer = Tokenizer() \
            .setInputCols(["document"]) \
            .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_healthcare_100d","en","clinical/models")\
        .setInputCols(["document","token"])\
        .setOutputCol("word_embeddings")

sentence_embeddings = SentenceEmbeddings() \
        .setInputCols(["document", "word_embeddings"]) \
        .setOutputCol("sentence_embeddings") \
        .setPoolingStrategy("AVERAGE")

embeddings_pipeline = Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        word_embeddings,
        sentence_embeddings,

    ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

embeddings_healthcare_100d download started this may take some time.
Approximate size to download 475.8 MB
[OK!]

In [25]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)
trainingData_with_embeddings = trainingData_with_embeddings.select("text","category","sentence_embeddings")
trainingData_with_embeddings.show(2,truncate=60)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------------------------+--------+------------------------------------------------------------+
|                                                        text|category|                                         sentence_embeddings|
+------------------------------------------------------------+--------+------------------------------------------------------------+
| "Syndrome malin"-like symptoms probably due to interacti...|     neg|[{sentence_embeddings, 0, 109,  "Syndrome malin"-like sym...|
| 'Bail-out' bivalirudin use in patients with thrombotic c...|     neg|[{sentence_embeddings, 0, 150,  'Bail-out' bivalirudin us...|
+------------------------------------------------------------+--------+------------------------------------------------------------+
only showing top 2 rows

In [26]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)
testData_with_embeddings = testData_with_embeddings.select("text","category","sentence_embeddings")
testData_with_embeddings.show(2,truncate=60)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------------------------+--------+------------------------------------------------------------+
|                                                        text|category|                                         sentence_embeddings|
+------------------------------------------------------------+--------+------------------------------------------------------------+
| 1 previously untreated patient with plasma-cell leukaemi...|     neg|[{sentence_embeddings, 0, 166,  1 previously untreated pa...|
| A 15-year-old boy had temporary hypertropia, supraductio...|     neg|[{sentence_embeddings, 0, 262,  A 15-year-old boy had tem...|
+------------------------------------------------------------+--------+------------------------------------------------------------+
only showing top 2 rows

In [27]:
testData_with_embeddings.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)

In [28]:
testData_with_embeddings.select(testData_with_embeddings.sentence_embeddings.embeddings).show(3,truncate=120)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------------------------------------------------------------------------------------+
|                                                                                          sentence_embeddings.embeddings|
+------------------------------------------------------------------------------------------------------------------------+
|[[0.022050034, -0.015850784, 0.11884041, 0.16476665, -0.14348501, -0.1576596, -0.031154968, -0.1393246, 0.008547665, ...|
|[[0.036828704, 0.052321058, 0.11382609, -0.05798419, -0.1450678, 0.13031518, 0.13105926, 0.077215254, -0.11359191, -0...|
|[[0.06133546, 0.08840095, 0.15639272, 0.114668004, -0.2140057, -0.08125059, 0.08913489, -0.1002837, 0.004876906, -0.0...|
+------------------------------------------------------------------------------------------------------------------------+
only showing top 3 rows

In [29]:
%%sh
hadoop fs -mkdir /tmp/ADE_logs_healthcare_100d

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/share/aws/emr/emrfs/lib/slf4j-log4j12-1.7.12.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]


In [30]:
%%sh
hadoop fs -ls /tmp/

Found 4 items
drwxr-xr-x   - emr-notebook hdfsadmingroup          0 2023-09-01 05:05 /tmp/ADE_logs_healthcare_100d
drwxrwxrwt   - yarn         hdfsadmingroup          0 2023-08-31 15:53 /tmp/entity-file-history
drwxrwxrwx   - mapred       mapred                  0 2023-08-31 15:43 /tmp/hadoop-yarn
drwxr-xr-x   - livy         hdfsadmingroup          0 2023-09-01 02:13 /tmp/livy


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/share/aws/emr/emrfs/lib/slf4j-log4j12-1.7.12.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]


In [31]:
log_folder="/tmp/ADE_logs_healthcare_100d"
# !mkdir -p $log_folder

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### ClassifierDL

In [32]:
classifier_dl = ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("prediction")\
        .setLabelColumn("category")\
        .setBatchSize(16)\
        .setMaxEpochs(30)\
        .setLr(0.002)\
        .setDropout(0.3)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)\
        # .setValidationSplit(0.1)

classifier_dl_pipeline = Pipeline(
    stages = [
        classifier_dl
    ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
clfDL_model_hc100 = classifier_dl_pipeline.fit(trainingData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
%%sh
hadoop fs -ls /tmp/ADE_logs_healthcare_100d/ClassifierDLApproach_*

-rw-r--r--   1 livy hdfsadmingroup       2199 2023-09-01 05:06 /tmp/ADE_logs_healthcare_100d/ClassifierDLApproach_721535809ab8.log


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/share/aws/emr/emrfs/lib/slf4j-log4j12-1.7.12.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]


In [35]:
%%sh
hadoop fs -cat /tmp/ADE_logs_healthcare_100d/ClassifierDLApproach_*

Training started - epochs: 30 - learning_rate: 0.002 - batch_size: 16 - training_examples: 5531 - classes: 2
Epoch 0/30 - 1.60s - loss: 187.70439 - acc: 0.759552 - batches: 346
Epoch 1/30 - 1.10s - loss: 176.14748 - acc: 0.8116271 - batches: 346
Epoch 2/30 - 1.10s - loss: 175.5478 - acc: 0.82938075 - batches: 346
Epoch 3/30 - 1.11s - loss: 174.23607 - acc: 0.8406126 - batches: 346
Epoch 4/30 - 1.08s - loss: 172.32031 - acc: 0.846772 - batches: 346
Epoch 5/30 - 1.10s - loss: 170.58084 - acc: 0.85456187 - batches: 346
Epoch 6/30 - 1.07s - loss: 169.14948 - acc: 0.8599967 - batches: 346
Epoch 7/30 - 1.10s - loss: 168.06482 - acc: 0.8623518 - batches: 346
Epoch 8/30 - 1.06s - loss: 167.17068 - acc: 0.8652503 - batches: 346
Epoch 9/30 - 1.08s - loss: 166.39902 - acc: 0.86869234 - batches: 346
Epoch 10/30 - 1.08s - loss: 165.6691 - acc: 0.8710474 - batches: 346
Epoch 11/30 - 1.10s - loss: 165.0651 - acc: 0.8723155 - batches: 346
Epoch 12/30 - 1.07s - loss: 164.51015 - acc: 0.8744894 - batche

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/share/aws/emr/emrfs/lib/slf4j-log4j12-1.7.12.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]


In [36]:
preds = clfDL_model_hc100.transform(testData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    

In [38]:
preds_df = preds.select("category","text","prediction.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

              precision    recall  f1-score   support

         neg       0.86      0.91      0.89       982
         pos       0.77      0.68      0.72       445

    accuracy                           0.84      1427
   macro avg       0.82      0.80      0.80      1427
weighted avg       0.83      0.84      0.84      1427

In [39]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df = pd.DataFrame(columns=["pos-f1-score","accuracy"])
results_df.loc["ClassifierDL_100d"] = [res["pos"]["f1-score"], res["accuracy"]]
results_df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                   pos-f1-score  accuracy
ClassifierDL_100d      0.724014  0.838122

In [40]:
%%display
spark.createDataFrame(results_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

### MultiClassifierDL

We will use MultiClassifierDL built by using Bidirectional GRU and CNNs inside TensorFlow that supports up to 100 classes. It is designed for multi-label classification purposes. Here we will use MultiClassifierDL as a binary classifier.

In [41]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
multiClassifier = MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(16)\
  .setMaxEpochs(15)\
  .setLr(9e-3)\
  .setThreshold(0.5)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = Pipeline(
    stages = [
        multiClassifier
    ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
multiClassifier_model_hc100 = multiClassifier_pipeline.fit(trainingData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
%%sh
hadoop fs -ls  /tmp/ADE_logs_healthcare_100d/ClassifierDLApproach_*

-rw-r--r--   1 livy hdfsadmingroup       2199 2023-09-01 05:06 /tmp/ADE_logs_healthcare_100d/ClassifierDLApproach_721535809ab8.log


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/share/aws/emr/emrfs/lib/slf4j-log4j12-1.7.12.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]


In [45]:
%%sh
hadoop fs -cat /tmp/ADE_logs_healthcare_100d/ClassifierDLApproach_*

Training started - epochs: 30 - learning_rate: 0.002 - batch_size: 16 - training_examples: 5531 - classes: 2
Epoch 0/30 - 1.60s - loss: 187.70439 - acc: 0.759552 - batches: 346
Epoch 1/30 - 1.10s - loss: 176.14748 - acc: 0.8116271 - batches: 346
Epoch 2/30 - 1.10s - loss: 175.5478 - acc: 0.82938075 - batches: 346
Epoch 3/30 - 1.11s - loss: 174.23607 - acc: 0.8406126 - batches: 346
Epoch 4/30 - 1.08s - loss: 172.32031 - acc: 0.846772 - batches: 346
Epoch 5/30 - 1.10s - loss: 170.58084 - acc: 0.85456187 - batches: 346
Epoch 6/30 - 1.07s - loss: 169.14948 - acc: 0.8599967 - batches: 346
Epoch 7/30 - 1.10s - loss: 168.06482 - acc: 0.8623518 - batches: 346
Epoch 8/30 - 1.06s - loss: 167.17068 - acc: 0.8652503 - batches: 346
Epoch 9/30 - 1.08s - loss: 166.39902 - acc: 0.86869234 - batches: 346
Epoch 10/30 - 1.08s - loss: 165.6691 - acc: 0.8710474 - batches: 346
Epoch 11/30 - 1.10s - loss: 165.0651 - acc: 0.8723155 - batches: 346
Epoch 12/30 - 1.07s - loss: 164.51015 - acc: 0.8744894 - batche

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/share/aws/emr/emrfs/lib/slf4j-log4j12-1.7.12.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]


In [46]:
preds = multiClassifier_model_hc100.transform(testData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category_array, preds.prediction.result).show(5, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [48]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['neg']           1120
['pos']            304
['pos', 'neg']       3
Name: result, dtype: int64

In [60]:
preds_df[preds_df.result.apply(len)==2]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Empty DataFrame
Columns: [category, text, result, metadata, scores]
Index: []

In [50]:
preds_df[preds_df.result.apply(len)==0]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Empty DataFrame
Columns: [category, text, result, metadata]
Index: []

MultiClassifierDL is a multi-label classifier, so some predictions may include both labels or none of the labels. That can be controlled a bit with `.setThreshold()` parameter during training. For now we will keep not keep zero label predictions and get the highest score as prediction.

In [51]:
# We will get the highest score label as result. You can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df["result"].value_counts()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

neg    1122
pos     305
Name: result, dtype: int64

In [52]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["MultiClassifierDL_100d"] = [res["pos"]["f1-score"], res["accuracy"]]

print (classification_report(preds_df["category"], preds_df["result"]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

              precision    recall  f1-score   support

         neg       0.81      0.93      0.87       982
         pos       0.77      0.53      0.63       445

    accuracy                           0.81      1427
   macro avg       0.79      0.73      0.75      1427
weighted avg       0.80      0.81      0.79      1427

### Generic Classifier

In [53]:
%%sh
wget -q -O /tmp/pet.in1202D.out2.pb https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/generic_classifier_graph/pet.in1202D.out2.pb

In [54]:
%%sh
hdfs dfs -copyFromLocal /tmp/pet.in1202D.out2.pb /user/hadoop/pet.in1202D.out2.pb


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/share/aws/emr/emrfs/lib/slf4j-log4j12-1.7.12.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]


In [72]:
%%sh
hdfs dfs -mkdir /tmp/gc_graph

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/share/aws/emr/emrfs/lib/slf4j-log4j12-1.7.12.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]


In [65]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "/tmp/gc_graph"

gc_graph_builder = TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_100d.pb")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# checkpoint

GenericClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericClassifierApproach takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [66]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

gen_clf = GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_100d.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(128)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: /tmp/gc_graph
Graph file name: gcf_graph_100d.pb
Build params: {'input_dim': 100, 'output_dim': 2, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to /tmp/gc_graph/gcf_graph_100d.pb

In [69]:
%%sh
hadoop fs -ls /tmp/ADE_logs_healthcare_100d/GenericClassifierApproach_*

-rw-r--r--   1 livy hdfsadmingroup       1244 2023-09-01 05:21 /tmp/ADE_logs_healthcare_100d/GenericClassifierApproach_a33c8c2d1c70.log


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/share/aws/emr/emrfs/lib/slf4j-log4j12-1.7.12.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]


In [70]:
%%sh
hadoop fs -cat /tmp/ADE_logs_healthcare_100d/GenericClassifierApproach_*

Training 25 epochs
Epoch 1/25	0.33s	Loss: 11.695651	ACC: 0.63477874
Epoch 2/25	0.11s	Loss: 9.9073305	ACC: 0.7231955
Epoch 3/25	0.11s	Loss: 9.622331	ACC: 0.7216435
Epoch 4/25	0.11s	Loss: 9.681353	ACC: 0.71903276
Epoch 5/25	0.10s	Loss: 9.264862	ACC: 0.7297717
Epoch 6/25	0.11s	Loss: 9.152498	ACC: 0.73593354
Epoch 7/25	0.10s	Loss: 8.780604	ACC: 0.75121003
Epoch 8/25	0.10s	Loss: 9.110051	ACC: 0.7426018
Epoch 9/25	0.10s	Loss: 8.797475	ACC: 0.7567143
Epoch 10/25	0.10s	Loss: 8.8014345	ACC: 0.74650806
Epoch 11/25	0.10s	Loss: 8.437761	ACC: 0.7709583
Epoch 12/25	0.10s	Loss: 8.442232	ACC: 0.76595384
Epoch 13/25	0.10s	Loss: 8.356702	ACC: 0.774687
Epoch 14/25	0.10s	Loss: 8.395545	ACC: 0.7653225
Epoch 15/25	0.10s	Loss: 8.446406	ACC: 0.76955104
Epoch 16/25	0.10s	Loss: 8.423889	ACC: 0.7600024
Epoch 17/25	0.10s	Loss: 8.152211	ACC: 0.7812566
Epoch 18/25	0.11s	Loss: 8.054671	ACC: 0.77694917
Epoch 19/25	0.11s	Loss: 7.924115	ACC: 0.78450525
Epoch 20/25	0.11s	Loss: 8.114472	ACC: 0.77766603
Epoch 21/25	0.11s	

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/share/aws/emr/emrfs/lib/slf4j-log4j12-1.7.12.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]


In [73]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [75]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print(classification_report(preds_df['category'], preds_df['result']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

              precision    recall  f1-score   support

         neg       0.92      0.72      0.81       982
         pos       0.58      0.86      0.70       445

    accuracy                           0.77      1427
   macro avg       0.75      0.79      0.75      1427
weighted avg       0.81      0.77      0.77      1427

In [76]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericClassifier_100d"] = [res["pos"]["f1-score"], res["accuracy"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [77]:
%%display
spark.createDataFrame(results_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

### GenericLogRegClassifier

In [78]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_logreg_graph_builder = TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [79]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(20)\
    .setBatchSize(128)\
    .setLearningRate(0.01)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [80]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 100, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb

In [81]:
%%sh
hadoop fs -cat /tmp/ADE_logs_healthcare_100d/GenericLogRegClassifierApproach*

Training 20 epochs
Epoch 1/20	0.12s	Loss: 26.117697	ACC: 0.7033157
Epoch 2/20	0.04s	Loss: 24.415691	ACC: 0.7176123
Epoch 3/20	0.04s	Loss: 23.327072	ACC: 0.7284828
Epoch 4/20	0.04s	Loss: 22.709171	ACC: 0.7401489
Epoch 5/20	0.05s	Loss: 22.515799	ACC: 0.7398924
Epoch 6/20	0.04s	Loss: 22.305206	ACC: 0.7501447
Epoch 7/20	0.05s	Loss: 22.293074	ACC: 0.74491006
Epoch 8/20	0.04s	Loss: 21.983974	ACC: 0.7458439
Epoch 9/20	0.04s	Loss: 21.70054	ACC: 0.75631315
Epoch 10/20	0.04s	Loss: 21.716377	ACC: 0.7541891
Epoch 11/20	0.04s	Loss: 21.55655	ACC: 0.75507027
Epoch 12/20	0.04s	Loss: 21.634256	ACC: 0.7591541
Epoch 13/20	0.04s	Loss: 21.356678	ACC: 0.75835174
Epoch 14/20	0.04s	Loss: 21.35975	ACC: 0.7615478
Epoch 15/20	0.04s	Loss: 21.41563	ACC: 0.75427455
Epoch 16/20	0.03s	Loss: 21.473177	ACC: 0.75596464
Epoch 17/20	0.03s	Loss: 21.556063	ACC: 0.7541891
Epoch 18/20	0.03s	Loss: 21.246853	ACC: 0.76354694
Epoch 19/20	0.03s	Loss: 21.287205	ACC: 0.7578257
Epoch 20/20	0.04s	Loss: 21.203638	ACC: 0.7605745
Trainin

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/share/aws/emr/emrfs/lib/slf4j-log4j12-1.7.12.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]


In [82]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [83]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [84]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

              precision    recall  f1-score   support

         neg       0.75      0.94      0.84       982
         pos       0.71      0.31      0.43       445

    accuracy                           0.74      1427
   macro avg       0.73      0.63      0.63      1427
weighted avg       0.74      0.74      0.71      1427

In [85]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericLogReg_100d"] = [res["pos"]["f1-score"], res["accuracy"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### GenericSVMClassifier

In [86]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_svm_graph_builder = TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [87]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(21)\
    .setBatchSize(128)\
    .setLearningRate(0.02)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [88]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 100, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb

In [89]:
%%sh
hadoop dfs -cat /tmp/ADE_logs_healthcare_100d/GenericSVMClassifierApproach*

Training 21 epochs
Epoch 1/21	0.14s	Loss: 28.011269	ACC: 0.70234245
Epoch 2/21	0.04s	Loss: 24.936155	ACC: 0.7231626
Epoch 3/21	0.04s	Loss: 24.16092	ACC: 0.73633474
Epoch 4/21	0.04s	Loss: 24.275904	ACC: 0.7429109
Epoch 5/21	0.04s	Loss: 24.135603	ACC: 0.7445944
Epoch 6/21	0.03s	Loss: 23.88545	ACC: 0.75595146
Epoch 7/21	0.04s	Loss: 24.120308	ACC: 0.75209785
Epoch 8/21	0.04s	Loss: 24.077988	ACC: 0.7570694
Epoch 9/21	0.04s	Loss: 23.975689	ACC: 0.7540049
Epoch 10/21	0.03s	Loss: 23.766499	ACC: 0.7608442
Epoch 11/21	0.04s	Loss: 24.056591	ACC: 0.7521833
Epoch 12/21	0.03s	Loss: 23.815363	ACC: 0.76354694
Epoch 13/21	0.03s	Loss: 23.821688	ACC: 0.76106775
Epoch 14/21	0.03s	Loss: 23.839157	ACC: 0.755695
Epoch 15/21	0.03s	Loss: 23.836073	ACC: 0.75573444
Epoch 16/21	0.03s	Loss: 23.594376	ACC: 0.7657697
Epoch 17/21	0.03s	Loss: 23.666729	ACC: 0.76306033
Epoch 18/21	0.04s	Loss: 23.440668	ACC: 0.7655461
Epoch 19/21	0.04s	Loss: 23.666946	ACC: 0.7551229
Epoch 20/21	0.03s	Loss: 24.085264	ACC: 0.7576481
Epoch


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/share/aws/emr/emrfs/lib/slf4j-log4j12-1.7.12.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]


In [90]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [91]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [92]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

              precision    recall  f1-score   support

         neg       0.75      0.95      0.84       982
         pos       0.72      0.29      0.41       445

    accuracy                           0.74      1427
   macro avg       0.73      0.62      0.62      1427
weighted avg       0.74      0.74      0.70      1427

In [93]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericSVM_100d"] = [res["pos"]["f1-score"], res["accuracy"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 200 Dimension Healthcare Embeddings (embeddings_clinical)



Now we will extract [embeddings_clinical](https://nlp.johnsnowlabs.com/2020/01/28/embeddings_clinical_en.html) embeddings which has 200 dimension output and use this embeddings in the model training.

In [94]:
document_assembler = DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

tokenizer = Tokenizer() \
            .setInputCols(["document"]) \
            .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical","en","clinical/models")\
        .setInputCols(["document","token"])\
        .setOutputCol("word_embeddings")

sentence_embeddings = SentenceEmbeddings() \
        .setInputCols(["document", "word_embeddings"]) \
        .setOutputCol("sentence_embeddings") \
        .setPoolingStrategy("AVERAGE")

embeddings_pipeline = Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        word_embeddings,
        sentence_embeddings,
    ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]

In [95]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)\
                                                  .select("text","category","sentence_embeddings")
trainingData_with_embeddings.show(2,truncate=60)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------------------------+--------+------------------------------------------------------------+
|                                                        text|category|                                         sentence_embeddings|
+------------------------------------------------------------+--------+------------------------------------------------------------+
| "Syndrome malin"-like symptoms probably due to interacti...|     neg|[{sentence_embeddings, 0, 109,  "Syndrome malin"-like sym...|
| 'Bail-out' bivalirudin use in patients with thrombotic c...|     neg|[{sentence_embeddings, 0, 150,  'Bail-out' bivalirudin us...|
+------------------------------------------------------------+--------+------------------------------------------------------------+
only showing top 2 rows

In [96]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)\
                                                  .select("text","category","sentence_embeddings")
testData_with_embeddings.show(2,truncate=60)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------------------------+--------+------------------------------------------------------------+
|                                                        text|category|                                         sentence_embeddings|
+------------------------------------------------------------+--------+------------------------------------------------------------+
| 1 previously untreated patient with plasma-cell leukaemi...|     neg|[{sentence_embeddings, 0, 166,  1 previously untreated pa...|
| A 15-year-old boy had temporary hypertropia, supraductio...|     neg|[{sentence_embeddings, 0, 262,  A 15-year-old boy had tem...|
+------------------------------------------------------------+--------+------------------------------------------------------------+
only showing top 2 rows

In [97]:
log_folder="ADE_logs_healthcare_200d"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### ClassifierDL

In [98]:
classifier_dl = ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("class")\
        .setLabelColumn("category")\
        .setBatchSize(16)\
        .setMaxEpochs(30)\
        .setLr(0.001)\
        .setDropout(0.3)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)

classifier_dl_pipeline = Pipeline(
    stages = [
        classifier_dl
    ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [99]:
clfDL_model_hc200 = classifier_dl_pipeline.fit(trainingData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [100]:
preds = clfDL_model_hc200.transform(testData_with_embeddings)

preds_df = preds.select("category","text","class.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

              precision    recall  f1-score   support

         neg       0.87      0.93      0.90       982
         pos       0.81      0.71      0.76       445

    accuracy                           0.86      1427
   macro avg       0.84      0.82      0.83      1427
weighted avg       0.86      0.86      0.85      1427

In [101]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["ClassifierDL_200d"] = [res["pos"]["f1-score"], res["accuracy"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### MultiClassifierDL

In [102]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [103]:
multiClassifier = MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(32)\
  .setMaxEpochs(20)\
  .setLr(9e-3)\
  .setThreshold(0.5)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = Pipeline(
    stages = [
        multiClassifier
    ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [104]:
multiClassifier_model_hc200 = multiClassifier_pipeline.fit(trainingData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [105]:
preds = multiClassifier_model_hc200.transform(testData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [106]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['neg']    1006
['pos']     421
Name: result, dtype: int64

In [107]:
# We will get the highest score label as result. You can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df["result"].value_counts()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

neg    1006
pos     421
Name: result, dtype: int64

In [108]:
print (classification_report(preds_df["category"], preds_df["result"]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

              precision    recall  f1-score   support

         neg       0.89      0.91      0.90       982
         pos       0.79      0.75      0.77       445

    accuracy                           0.86      1427
   macro avg       0.84      0.83      0.83      1427
weighted avg       0.86      0.86      0.86      1427

In [109]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["MultiClassifierDL_200d"] = [res["pos"]["f1-score"], res["accuracy"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Generic Classifier

In [110]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder = TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_200d.pb")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [111]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

gen_clf = GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_200d.pb")\
    .setEpochsNumber(40)\
    .setBatchSize(16)\
    .setLearningRate(0.001)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.5)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [112]:
generic_model_hc200 = clf_Pipeline.fit(trainingData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_200d.pb
Build params: {'input_dim': 200, 'output_dim': 2, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_200d.pb

In [113]:
pred_df = generic_model_hc200.transform(testData_with_embeddings)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

              precision    recall  f1-score   support

         neg       0.92      0.83      0.87       982
         pos       0.69      0.85      0.76       445

    accuracy                           0.83      1427
   macro avg       0.81      0.84      0.82      1427
weighted avg       0.85      0.83      0.84      1427

In [114]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericClassifier_200d"] = [res["pos"]["f1-score"], res["accuracy"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### GenericLogRegClassifier

In [115]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_logreg_graph_builder = TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [116]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(20)\
    .setBatchSize(128)\
    .setLearningRate(0.01)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [117]:
generic_model_hc200 = clf_Pipeline.fit(trainingData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 200, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb

In [118]:
pred_df = generic_model_hc200.transform(testData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [119]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

              precision    recall  f1-score   support

         neg       0.81      0.92      0.86       982
         pos       0.75      0.52      0.62       445

    accuracy                           0.80      1427
   macro avg       0.78      0.72      0.74      1427
weighted avg       0.79      0.80      0.79      1427

In [120]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericLogReg_200d"] = [res["pos"]["f1-score"], res["accuracy"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### GenericSVMClassifier

In [121]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_svm_graph_builder = TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [122]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(20)\
    .setBatchSize(128)\
    .setLearningRate(0.005)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [123]:
generic_model_hc200 = clf_Pipeline.fit(trainingData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 200, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb

In [124]:
pred_df = generic_model_hc200.transform(testData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [125]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

              precision    recall  f1-score   support

         neg       0.80      0.93      0.86       982
         pos       0.76      0.47      0.58       445

    accuracy                           0.79      1427
   macro avg       0.78      0.70      0.72      1427
weighted avg       0.78      0.79      0.77      1427

In [126]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericSVM_200d"] = [res["pos"]["f1-score"], res["accuracy"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Bert Sentence Embeddings (sbiobert_base_cased_mli)

Now we will extract [sbiobert_base_cased_mli](https://nlp.johnsnowlabs.com/2020/11/27/sbiobert_base_cased_mli_en.html) embeddings which has 768 dimension output and use this embeddings in the model training.

In [127]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

bert_sent = BertSentenceEmbeddings.pretrained("sbiobert_base_cased_mli", 'en','clinical/models')\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

embeddings_pipeline = Pipeline(
    stages = [document_assembler,
              bert_sent])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]

In [128]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)\
                                                  .select("text","category","sentence_embeddings")

trainingData_with_embeddings.show(2,truncate=60)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------------------------+--------+------------------------------------------------------------+
|                                                        text|category|                                         sentence_embeddings|
+------------------------------------------------------------+--------+------------------------------------------------------------+
| "Syndrome malin"-like symptoms probably due to interacti...|     neg|[{sentence_embeddings, 0, 109,  "Syndrome malin"-like sym...|
| 'Bail-out' bivalirudin use in patients with thrombotic c...|     neg|[{sentence_embeddings, 0, 150,  'Bail-out' bivalirudin us...|
+------------------------------------------------------------+--------+------------------------------------------------------------+
only showing top 2 rows

In [129]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)\
                                                  .select("text","category","sentence_embeddings")

testData_with_embeddings.show(2,truncate=60)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------------------------+--------+------------------------------------------------------------+
|                                                        text|category|                                         sentence_embeddings|
+------------------------------------------------------------+--------+------------------------------------------------------------+
| 1 previously untreated patient with plasma-cell leukaemi...|     neg|[{sentence_embeddings, 0, 166,  1 previously untreated pa...|
| A 15-year-old boy had temporary hypertropia, supraductio...|     neg|[{sentence_embeddings, 0, 262,  A 15-year-old boy had tem...|
+------------------------------------------------------------+--------+------------------------------------------------------------+
only showing top 2 rows

In [130]:
log_folder="ADE_logs_bert"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### ClassifierDL

In [131]:
classifier_dl = ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("class")\
        .setLabelColumn("category")\
        .setBatchSize(32)\
        .setMaxEpochs(20)\
        .setLr(0.001)\
        .setDropout(0.3)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)

classifier_dl_pipeline = Pipeline(stages=[classifier_dl])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [132]:
clfDL_model_bert = classifier_dl_pipeline.fit(trainingData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [133]:
preds = clfDL_model_bert.transform(testData_with_embeddings)

preds_df = preds.select("category","text","class.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

              precision    recall  f1-score   support

         neg       0.89      0.91      0.90       982
         pos       0.78      0.75      0.76       445

    accuracy                           0.86      1427
   macro avg       0.84      0.83      0.83      1427
weighted avg       0.86      0.86      0.86      1427

In [134]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["ClassifierDL_bert"] = [res["pos"]["f1-score"], res["accuracy"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### MultiClassifierDL

In [135]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [136]:
multiClassifier = MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(32)\
  .setMaxEpochs(20)\
  .setLr(9e-3)\
  .setThreshold(0.5)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = Pipeline(
    stages = [
        multiClassifier
    ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [137]:
multiClassifier_model_bert = multiClassifier_pipeline.fit(trainingData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [138]:
preds = multiClassifier_model_bert.transform(testData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [139]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['neg']    1022
['pos']     404
[]            1
Name: result, dtype: int64

In [140]:
# We will get the highest score label as result. You can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df = preds_df[(preds_df["result"]!="")]
preds_df["result"].value_counts()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

neg    1022
pos     404
Name: result, dtype: int64

In [141]:
print (classification_report(preds_df["category"], preds_df["result"]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

              precision    recall  f1-score   support

         neg       0.89      0.93      0.91       981
         pos       0.83      0.75      0.79       445

    accuracy                           0.87      1426
   macro avg       0.86      0.84      0.85      1426
weighted avg       0.87      0.87      0.87      1426

In [142]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["MultiClassifierDL_bert"] = [res["pos"]["f1-score"], res["accuracy"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Generic Classifier

In [143]:
# !pip install -q tensorflow==2.11.0 tensorflow_addons

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [144]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder = TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_bert.pb")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [145]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

gen_clf = GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_bert.pb")\
    .setEpochsNumber(20)\
    .setBatchSize(32)\
    .setLearningRate(0.001)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.5)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
generic_model_bert = clf_Pipeline.fit(trainingData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_bert.pb
Build params: {'input_dim': 768, 'output_dim': 2, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_bert.pb

In [ ]:
pred_df = generic_model_bert.transform(testData_with_embeddings)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

              precision    recall  f1-score   support

         neg       0.93      0.85      0.89       982
         pos       0.72      0.87      0.79       445

    accuracy                           0.85      1427
   macro avg       0.83      0.86      0.84      1427
weighted avg       0.87      0.85      0.86      1427

In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericClassifier_bert"] = [res["pos"]["f1-score"], res["accuracy"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### GenericLogRegClassifier

In [ ]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_logreg_graph_builder = TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(20)\
    .setBatchSize(128)\
    .setLearningRate(0.01)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
generic_model_bert = clf_Pipeline.fit(trainingData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 768, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb

In [ ]:
pred_df = generic_model_bert.transform(testData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

              precision    recall  f1-score   support

         neg       0.91      0.90      0.90       982
         pos       0.78      0.80      0.79       445

    accuracy                           0.87      1427
   macro avg       0.85      0.85      0.85      1427
weighted avg       0.87      0.87      0.87      1427

In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericLogReg_bert"] = [res["pos"]["f1-score"], res["accuracy"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### GenericSVMClassifier

In [ ]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_svm_graph_builder = TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(20)\
    .setBatchSize(128)\
    .setLearningRate(0.005)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
generic_model_bert = clf_Pipeline.fit(trainingData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 768, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb

In [ ]:
pred_df = generic_model_bert.transform(testData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

              precision    recall  f1-score   support

         neg       0.92      0.88      0.90       982
         pos       0.76      0.82      0.79       445

    accuracy                           0.86      1427
   macro avg       0.84      0.85      0.85      1427
weighted avg       0.87      0.86      0.87      1427

In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericSVM_bert"] = [res["pos"]["f1-score"], res["accuracy"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## DocumentLogRegClassifier

In [ ]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

normalizer = Normalizer() \
      .setInputCols(["token"]) \
      .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

stemmer = Stemmer() \
      .setInputCols(["cleanTokens"]) \
      .setOutputCol("stem")

logreg = DocumentLogRegClassifierApproach()\
    .setInputCols("stem")\
    .setLabelCol("category")\
    .setOutputCol("prediction")\
    .setMaxIter(10)\
    .setTol(1e-6)

clf_Pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    normalizer,
    stopwords_cleaner,
    stemmer,
    logreg])
doclogreg_model = clf_Pipeline.fit(trainingData)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
pred_df = doclogreg_model.transform(testData)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

              precision    recall  f1-score   support

         neg       0.85      0.87      0.86       982
         pos       0.70      0.67      0.68       445

    accuracy                           0.81      1427
   macro avg       0.78      0.77      0.77      1427
weighted avg       0.81      0.81      0.81      1427

In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["DocumentLogRegClassifier"] = [res["pos"]["f1-score"], res["accuracy"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Comparision of All Models' Result in ADE Dataset

Here is the all results of classifier models. Only positive ADE f1 scores and accuracy scores are presented.

In [ ]:
results_df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                          pos-f1-score  accuracy
ClassifierDL_100d             0.724014  0.838122
MultiClassifierDL_100d        0.629333  0.805186
GenericClassifier_100d        0.695731  0.765242
GenericLogReg_100d            0.431250  0.744919
GenericSVM_100d               0.413462  0.743518
ClassifierDL_200d             0.756303  0.857744
MultiClassifierDL_200d        0.766744  0.858444
GenericClassifier_200d        0.760563  0.833217
GenericLogReg_200d            0.615180  0.797477
GenericSVM_200d               0.582524  0.789068
ClassifierDL_bert             0.764977  0.857043
MultiClassifierDL_bert        0.789164  0.874474
GenericClassifier_bert        0.786585  0.852838
GenericLogReg_bert            0.792913  0.868956
GenericSVM_bert               0.791802  0.864751
DocumentLogRegClassifier      0.683544  0.807288

# Mtsamples Dataset

## Load Dataset

In [ ]:
%%sh
wget -q -O /tmp/mtsamples_classifier.csv https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/mtsamples_classifier.csv

In [ ]:
%%sh
hadoop fs -copyFromLocal /tmp/mtsamples_classifier.csv /user/hadoop/

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/share/aws/emr/emrfs/lib/slf4j-log4j12-1.7.12.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]


In [ ]:
%%sh
hadoop fs -ls /user/hadoop/

Found 7 items
-rw-r--r--   1 emr-notebook hdfsadmingroup     244358 2023-08-31 17:11 /user/hadoop/NER_NCBIconlltest.txt
-rw-r--r--   1 emr-notebook hdfsadmingroup    1091488 2023-08-31 17:11 /user/hadoop/NER_NCBIconlltrain.txt
-rw-r--r--   1 emr-notebook hdfsadmingroup     123124 2023-09-01 00:55 /user/hadoop/i2b2_assertion_sample_short.csv
-rw-r--r--   1 emr-notebook hdfsadmingroup     285456 2023-09-01 02:34 /user/hadoop/i2b2_clinical_rel_dataset.csv
-rw-r--r--   1 emr-notebook hdfsadmingroup    2079279 2023-09-01 06:32 /user/hadoop/mtsamples_classifier.csv
-rw-r--r--   1 emr-notebook hdfsadmingroup     122582 2023-09-01 05:08 /user/hadoop/pet.in1202D.out2.pb
-rw-r--r--   1 emr-notebook hdfsadmingroup    1796209 2023-08-31 17:07 /user/hadoop/tfgraphs


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/share/aws/emr/emrfs/lib/slf4j-log4j12-1.7.12.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]


In [ ]:
spark_df = spark.read.csv("/user/hadoop/mtsamples_classifier.csv", header = True)

spark_df.show(10,truncate=100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+----------------------------------------------------------------------------------------------------+
|        category|                                                                                                text|
+----------------+----------------------------------------------------------------------------------------------------+
|Gastroenterology| PROCEDURES PERFORMED: Colonoscopy. INDICATIONS: Renewed symptoms likely consistent with active f...|
|Gastroenterology| OPERATION 1. Ivor-Lewis esophagogastrectomy. 2. Feeding jejunostomy. 3. Placement of two right-s...|
|Gastroenterology| PREOPERATIVE DIAGNOSES: 1. Gastroesophageal reflux disease. 2. Chronic dyspepsia. POSTOPERATIVE ...|
|Gastroenterology| PROCEDURE: Colonoscopy. PREOPERATIVE DIAGNOSES: Rectal bleeding and perirectal abscess. POSTOPER...|
|Gastroenterology| PREOPERATIVE DIAGNOSIS: Right colon tumor. POSTOPERATIVE DIAGNOSES: 1. Right colon cancer. 2. As...|
|Gastroenterology| PREOPERATIVE DIAGNOSI

In [191]:
spark_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- category: string (nullable = true)
 |-- text: string (nullable = true)

In [192]:
spark_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

638

In [193]:
spark_df.groupBy("category").count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-----+
|        category|count|
+----------------+-----+
|         Urology|  115|
|      Orthopedic|  223|
|Gastroenterology|  157|
|       Neurology|  143|
+----------------+-----+

In [194]:
(trainingData, testData) = spark_df.randomSplit([0.8, 0.2], seed = 42)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [195]:
trainingData.groupBy("category").count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-----+
|        category|count|
+----------------+-----+
|         Urology|   95|
|      Orthopedic|  188|
|Gastroenterology|  132|
|       Neurology|  121|
+----------------+-----+

In [196]:
testData.groupBy("category").count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-----+
|        category|count|
+----------------+-----+
|         Urology|   20|
|      Orthopedic|   35|
|Gastroenterology|   25|
|       Neurology|   22|
+----------------+-----+

## 100 Dimension Healthcare Embeddings (embeddings_healthcare_100d)



We will extract [healthcare_100d embeddings](https://nlp.johnsnowlabs.com/2020/05/29/embeddings_healthcare_100d_en.html) and use it in the classificaiton model training.

In [197]:
document_assembler = DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

tokenizer = Tokenizer() \
            .setInputCols(["document"]) \
            .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_healthcare_100d","en","clinical/models")\
        .setInputCols(["document","token"])\
        .setOutputCol("word_embeddings")

sentence_embeddings = SentenceEmbeddings() \
        .setInputCols(["document", "word_embeddings"]) \
        .setOutputCol("sentence_embeddings") \
        .setPoolingStrategy("AVERAGE")

embeddings_pipeline = Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        word_embeddings,
        sentence_embeddings,

    ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

embeddings_healthcare_100d download started this may take some time.
Approximate size to download 475.8 MB
[OK!]

In [198]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)
trainingData_with_embeddings = trainingData_with_embeddings.select("text","category","sentence_embeddings")
trainingData_with_embeddings.show(2,truncate=60)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------------------------+----------------+------------------------------------------------------------+
|                                                        text|        category|                                         sentence_embeddings|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
| ADMISSION DIAGNOSIS: Symptomatic cholelithiasis. DISCHAR...|Gastroenterology|[{sentence_embeddings, 0, 2228,  ADMISSION DIAGNOSIS: Sym...|
| ADMITTING DIAGNOSES: Hiatal hernia, gastroesophageal ref...|Gastroenterology|[{sentence_embeddings, 0, 3237,  ADMITTING DIAGNOSES: Hia...|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
only showing top 2 rows

In [199]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)
testData_with_embeddings = testData_with_embeddings.select("text","category","sentence_embeddings")
testData_with_embeddings.show(2,truncate=60)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------------------------+----------------+------------------------------------------------------------+
|                                                        text|        category|                                         sentence_embeddings|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
| ADMITTING DIAGNOSIS: Gastrointestinal bleed. HISTORY OF ...|Gastroenterology|[{sentence_embeddings, 0, 3978,  ADMITTING DIAGNOSIS: Gas...|
| CHIEF COMPLAINT: Dysphagia and hematemesis while vomitin...|Gastroenterology|[{sentence_embeddings, 0, 6515,  CHIEF COMPLAINT: Dysphag...|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
only showing top 2 rows

In [200]:
testData_with_embeddings.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)

In [201]:
testData_with_embeddings.select(testData_with_embeddings.sentence_embeddings.embeddings).show(3,truncate=120)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------------------------------------------------------------------------------------+
|                                                                                          sentence_embeddings.embeddings|
+------------------------------------------------------------------------------------------------------------------------+
|[[-0.010433162, 0.0127568655, 0.110687375, 0.1609855, -0.14818177, -0.050856464, 0.064748034, -0.019294674, -0.002178...|
|[[-0.0055608996, 0.018396137, 0.12305377, 0.13861515, -0.18651922, -0.07559964, 0.12978752, -0.0046452526, -0.0031408...|
|[[-0.019461514, 0.060639214, 0.11142021, 0.12826534, -0.16216077, -0.10416892, 0.17136306, -0.020363769, 0.01169337, ...|
+------------------------------------------------------------------------------------------------------------------------+
only showing top 3 rows

In [202]:
log_folder="Mt_logs_healthcare_100d"
# !mkdir -p $log_folder

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### ClassifierDL

In [203]:
classifier_dl = ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("prediction")\
        .setLabelColumn("category")\
        .setBatchSize(8)\
        .setMaxEpochs(50)\
        .setLr(0.005)\
        .setDropout(0.3)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)\
        # .setValidationSplit(0.1)

classifier_dl_pipeline = Pipeline(stages=[classifier_dl])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [204]:
# to list all logs file , uncommand and run next cell

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [205]:
# %%sh
# hadoop fs -ls -R /user/livy

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [206]:
clfDL_model_hc100 = classifier_dl_pipeline.fit(trainingData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [207]:
preds = clfDL_model_hc100.transform(testData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [208]:
preds_df = preds.select("category","text","prediction.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                  precision    recall  f1-score   support

Gastroenterology       0.61      0.88      0.72        25
       Neurology       0.59      0.77      0.67        22
      Orthopedic       0.89      0.94      0.92        35
         Urology       0.00      0.00      0.00        20

        accuracy                           0.71       102
       macro avg       0.52      0.65      0.58       102
    weighted avg       0.58      0.71      0.64       102

In [209]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df = pd.DataFrame(columns=["macro-f1-score","weighted-f1-score","accuracy"])
results_df.loc["ClassifierDL_100d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]
results_df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                   macro-f1-score  weighted-f1-score  accuracy
ClassifierDL_100d        0.576161           0.635125  0.705882

### MultiClassifierDL

In [210]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [211]:
multiClassifier = MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(8)\
  .setMaxEpochs(40)\
  .setLr(5e-3)\
  .setThreshold(0.5)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = Pipeline(
    stages = [
        multiClassifier
    ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [212]:
multiClassifier_model_hc100 = multiClassifier_pipeline.fit(trainingData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [213]:
preds = multiClassifier_model_hc100.transform(testData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [214]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Orthopedic']                        35
['Urology']                           21
['Gastroenterology']                  20
['Neurology']                         17
[]                                     4
['Orthopedic', 'Neurology']            4
['Orthopedic', 'Gastroenterology']     1
Name: result, dtype: int64

In [215]:
# We will get the highest score label as result. You can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df = preds_df[(preds_df["result"]!="")]
preds_df["result"].value_counts()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Orthopedic          36
Gastroenterology    21
Urology             21
Neurology           20
Name: result, dtype: int64

In [216]:
print (classification_report(preds_df["category"], preds_df["result"]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                  precision    recall  f1-score   support

Gastroenterology       0.95      0.87      0.91        23
       Neurology       0.80      0.76      0.78        21
      Orthopedic       0.89      0.91      0.90        35
         Urology       0.90      1.00      0.95        19

        accuracy                           0.89        98
       macro avg       0.89      0.89      0.89        98
    weighted avg       0.89      0.89      0.89        98

In [217]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["MultiClassifierDL_100d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Generic Classifier

In [218]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder = TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_100d.pb")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [219]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

gen_clf = GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_100d.pb")\
    .setEpochsNumber(50)\
    .setBatchSize(32)\
    .setLearningRate(0.001)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.3)\
    .setFixImbalance(False)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [220]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_100d.pb
Build params: {'input_dim': 100, 'output_dim': 4, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_100d.pb

In [221]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [222]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                  precision    recall  f1-score   support

Gastroenterology       0.89      0.96      0.92        25
       Neurology       0.85      0.77      0.81        22
      Orthopedic       0.89      0.91      0.90        35
         Urology       0.95      0.90      0.92        20

        accuracy                           0.89       102
       macro avg       0.89      0.89      0.89       102
    weighted avg       0.89      0.89      0.89       102

In [223]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericClassifier_100d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### GenericLogRegClassifier

In [224]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_logreg_graph_builder = TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [225]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(45)\
    .setBatchSize(32)\
    .setLearningRate(0.02)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [226]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 100, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb

In [227]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [228]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                  precision    recall  f1-score   support

Gastroenterology       0.95      0.84      0.89        25
       Neurology       0.81      0.77      0.79        22
      Orthopedic       0.84      0.91      0.88        35
         Urology       0.81      0.85      0.83        20

        accuracy                           0.85       102
       macro avg       0.85      0.84      0.85       102
    weighted avg       0.86      0.85      0.85       102

In [229]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericLogReg_100d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### GenericSVMClassifier

In [230]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_svm_graph_builder = TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [231]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(45)\
    .setBatchSize(32)\
    .setLearningRate(0.02)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [232]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 100, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb

In [233]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [234]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                  precision    recall  f1-score   support

Gastroenterology       0.88      0.88      0.88        25
       Neurology       0.85      0.77      0.81        22
      Orthopedic       0.86      0.91      0.89        35
         Urology       0.85      0.85      0.85        20

        accuracy                           0.86       102
       macro avg       0.86      0.85      0.86       102
    weighted avg       0.86      0.86      0.86       102

In [235]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericSVM_100d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 200 Dimension Healthcare Embeddings (embeddings_clinical)



Now we will extract [embeddings_clinical](https://nlp.johnsnowlabs.com/2020/01/28/embeddings_clinical_en.html) embeddings which has 200 dimension output and use this embeddings in the model training.

In [236]:
document_assembler = DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

tokenizer = Tokenizer() \
            .setInputCols(["document"]) \
            .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical","en","clinical/models")\
        .setInputCols(["document","token"])\
        .setOutputCol("word_embeddings")

sentence_embeddings = SentenceEmbeddings() \
        .setInputCols(["document", "word_embeddings"]) \
        .setOutputCol("sentence_embeddings") \
        .setPoolingStrategy("AVERAGE")

embeddings_pipeline = Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        word_embeddings,
        sentence_embeddings,
    ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]

In [237]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)\
                                                  .select("text","category","sentence_embeddings")
trainingData_with_embeddings.show(2,truncate=60)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------------------------+----------------+------------------------------------------------------------+
|                                                        text|        category|                                         sentence_embeddings|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
| ADMISSION DIAGNOSIS: Symptomatic cholelithiasis. DISCHAR...|Gastroenterology|[{sentence_embeddings, 0, 2228,  ADMISSION DIAGNOSIS: Sym...|
| ADMITTING DIAGNOSES: Hiatal hernia, gastroesophageal ref...|Gastroenterology|[{sentence_embeddings, 0, 3237,  ADMITTING DIAGNOSES: Hia...|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
only showing top 2 rows

In [238]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)\
                                                  .select("text","category","sentence_embeddings")
testData_with_embeddings.show(2,truncate=60)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------------------------+----------------+------------------------------------------------------------+
|                                                        text|        category|                                         sentence_embeddings|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
| ADMITTING DIAGNOSIS: Gastrointestinal bleed. HISTORY OF ...|Gastroenterology|[{sentence_embeddings, 0, 3978,  ADMITTING DIAGNOSIS: Gas...|
| CHIEF COMPLAINT: Dysphagia and hematemesis while vomitin...|Gastroenterology|[{sentence_embeddings, 0, 6515,  CHIEF COMPLAINT: Dysphag...|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
only showing top 2 rows

In [239]:
log_folder="Mt_logs_healthcare_200d"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### ClassifierDL

In [240]:
classifier_dl = ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("class")\
        .setLabelColumn("category")\
        .setBatchSize(32)\
        .setMaxEpochs(30)\
        .setLr(0.01)\
        .setDropout(0.1)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)

classifier_dl_pipeline = Pipeline(stages=[classifier_dl])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [241]:
clfDL_model_hc200 = classifier_dl_pipeline.fit(trainingData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [242]:
preds = clfDL_model_hc200.transform(testData_with_embeddings)

preds_df = preds.select("category","text","class.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                  precision    recall  f1-score   support

Gastroenterology       0.00      0.00      0.00        25
       Neurology       0.00      0.00      0.00        22
      Orthopedic       0.34      1.00      0.51        35
         Urology       0.00      0.00      0.00        20

        accuracy                           0.34       102
       macro avg       0.09      0.25      0.13       102
    weighted avg       0.12      0.34      0.18       102

In [243]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["ClassifierDL_200d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### MultiClassifierDL



In [244]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [245]:
multiClassifier = MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(16)\
  .setMaxEpochs(20)\
  .setLr(9e-3)\
  .setThreshold(0.2)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = Pipeline(
    stages = [
        multiClassifier
    ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [246]:
multiClassifier_model_hc200 = multiClassifier_pipeline.fit(trainingData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [247]:
preds = multiClassifier_model_hc200.transform(testData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [248]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Orthopedic']                        32
['Gastroenterology']                  19
['Urology']                           17
['Neurology']                         15
['Orthopedic', 'Neurology']            8
['Orthopedic', 'Gastroenterology']     4
[]                                     4
['Urology', 'Gastroenterology']        2
['Urology', 'Orthopedic']              1
Name: result, dtype: int64

In [249]:
# We will get the highest score label as result. you can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df = preds_df[(preds_df["result"]!="")]
preds_df["result"].value_counts()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Orthopedic          38
Gastroenterology    22
Urology             20
Neurology           18
Name: result, dtype: int64

In [250]:
print (classification_report(preds_df["category"], preds_df["result"]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                  precision    recall  f1-score   support

Gastroenterology       1.00      0.92      0.96        24
       Neurology       0.89      0.80      0.84        20
      Orthopedic       0.87      0.94      0.90        35
         Urology       0.90      0.95      0.92        19

        accuracy                           0.91        98
       macro avg       0.91      0.90      0.91        98
    weighted avg       0.91      0.91      0.91        98

In [251]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["MultiClassifierDL_200d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Generic Classifier

In [252]:
# !pip install -q tensorflow==2.7.0 tensorflow_addons

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [253]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder = TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_200d.pb")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [254]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

gen_clf = GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_200d.pb")\
    .setEpochsNumber(30)\
    .setBatchSize(32)\
    .setLearningRate(0.001)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.3)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [255]:
generic_model_hc200 = clf_Pipeline.fit(trainingData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_200d.pb
Build params: {'input_dim': 200, 'output_dim': 4, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_200d.pb

In [256]:
pred_df = generic_model_hc200.transform(testData_with_embeddings)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                  precision    recall  f1-score   support

Gastroenterology       0.91      0.84      0.87        25
       Neurology       0.88      0.68      0.77        22
      Orthopedic       0.87      0.94      0.90        35
         Urology       0.79      0.95      0.86        20

        accuracy                           0.86       102
       macro avg       0.86      0.85      0.85       102
    weighted avg       0.87      0.86      0.86       102

In [257]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericClassifier_200d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### GenericLogRegClassifier

In [258]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_logreg_graph_builder = TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [259]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(45)\
    .setBatchSize(32)\
    .setLearningRate(0.02)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [260]:
generic_model_hc200 = clf_Pipeline.fit(trainingData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 200, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb

In [261]:
pred_df = generic_model_hc200.transform(testData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [262]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                  precision    recall  f1-score   support

Gastroenterology       0.84      0.84      0.84        25
       Neurology       0.89      0.73      0.80        22
      Orthopedic       0.85      0.94      0.89        35
         Urology       0.80      0.80      0.80        20

        accuracy                           0.84       102
       macro avg       0.84      0.83      0.83       102
    weighted avg       0.84      0.84      0.84       102

In [263]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericLogReg_200d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### GenericSVMClassifier

In [264]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_svm_graph_builder = TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [265]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(45)\
    .setBatchSize(32)\
    .setLearningRate(0.02)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [266]:
generic_model_hc200 = clf_Pipeline.fit(trainingData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 200, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb

In [267]:
pred_df = generic_model_hc200.transform(testData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [268]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                  precision    recall  f1-score   support

Gastroenterology       0.79      0.92      0.85        25
       Neurology       0.78      0.82      0.80        22
      Orthopedic       0.88      0.86      0.87        35
         Urology       0.94      0.75      0.83        20

        accuracy                           0.84       102
       macro avg       0.85      0.84      0.84       102
    weighted avg       0.85      0.84      0.84       102

In [269]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericSVM_200d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Bert Sentence Embeddings (sbiobert_base_cased_mli)

We will extract [sbiobert_base_cased_mli](https://nlp.johnsnowlabs.com/2020/11/27/sbiobert_base_cased_mli_en.html) embeddings which has 768 dimension output and use this embeddings in the model training.

In [270]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

bert_sent = BertSentenceEmbeddings.pretrained("sbiobert_base_cased_mli", 'en','clinical/models')\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

embeddings_pipeline = Pipeline(
    stages = [document_assembler,
              bert_sent])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]

In [271]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)\
                                                  .select("text","category","sentence_embeddings")

trainingData_with_embeddings.show(2,truncate=60)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------------------------+----------------+------------------------------------------------------------+
|                                                        text|        category|                                         sentence_embeddings|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
| ADMISSION DIAGNOSIS: Symptomatic cholelithiasis. DISCHAR...|Gastroenterology|[{sentence_embeddings, 0, 2228,  ADMISSION DIAGNOSIS: Sym...|
| ADMITTING DIAGNOSES: Hiatal hernia, gastroesophageal ref...|Gastroenterology|[{sentence_embeddings, 0, 3237,  ADMITTING DIAGNOSES: Hia...|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
only showing top 2 rows

In [272]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)\
                                                  .select("text","category","sentence_embeddings")

testData_with_embeddings.show(2,truncate=60)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------------------------+----------------+------------------------------------------------------------+
|                                                        text|        category|                                         sentence_embeddings|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
| ADMITTING DIAGNOSIS: Gastrointestinal bleed. HISTORY OF ...|Gastroenterology|[{sentence_embeddings, 0, 3978,  ADMITTING DIAGNOSIS: Gas...|
| CHIEF COMPLAINT: Dysphagia and hematemesis while vomitin...|Gastroenterology|[{sentence_embeddings, 0, 6515,  CHIEF COMPLAINT: Dysphag...|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
only showing top 2 rows

In [273]:
log_folder="Mt_logs_bert"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### ClassifierDL

In [274]:
classifier_dl = ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("class")\
        .setLabelColumn("category")\
        .setBatchSize(8)\
        .setMaxEpochs(30)\
        .setLr(0.002)\
        .setDropout(0.1)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)

classifier_dl_pipeline = Pipeline(stages=[classifier_dl])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [275]:
clfDL_model_bert = classifier_dl_pipeline.fit(trainingData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [276]:
preds = clfDL_model_bert.transform(testData_with_embeddings)

preds_df = preds.select("category","text","class.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                  precision    recall  f1-score   support

Gastroenterology       0.92      0.92      0.92        25
       Neurology       0.82      0.82      0.82        22
      Orthopedic       0.88      0.86      0.87        35
         Urology       0.90      0.95      0.93        20

        accuracy                           0.88       102
       macro avg       0.88      0.89      0.88       102
    weighted avg       0.88      0.88      0.88       102

In [277]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["ClassifierDL_bert"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### MultiClassifierDL

In [278]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [279]:
multiClassifier = MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(8)\
  .setMaxEpochs(20)\
  .setLr(9e-3)\
  .setThreshold(0.5)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = Pipeline(
    stages = [
        multiClassifier
    ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [280]:
multiClassifier_model_bert = multiClassifier_pipeline.fit(trainingData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [281]:
preds = multiClassifier_model_bert.transform(testData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [282]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Orthopedic']                 32
['Gastroenterology']           24
['Neurology']                  21
['Urology']                    19
[]                              5
['Orthopedic', 'Neurology']     1
Name: result, dtype: int64

In [283]:
# We will get the highest score label as result. You can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df = preds_df[(preds_df["result"]!="")]
preds_df["result"].value_counts()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Orthopedic          33
Gastroenterology    24
Neurology           21
Urology             19
Name: result, dtype: int64

In [284]:
print (classification_report(preds_df["category"], preds_df["result"]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                  precision    recall  f1-score   support

Gastroenterology       0.96      0.92      0.94        25
       Neurology       0.81      0.85      0.83        20
      Orthopedic       0.91      0.86      0.88        35
         Urology       0.84      0.94      0.89        17

        accuracy                           0.89        97
       macro avg       0.88      0.89      0.88        97
    weighted avg       0.89      0.89      0.89        97

In [285]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["MultiClassifierDL_bert"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Generic Classifier

In [286]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder = TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_bert.pb")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [287]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

gen_clf = GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_bert.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(16)\
    .setLearningRate(0.002)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [288]:
generic_model_bert = clf_Pipeline.fit(trainingData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_bert.pb
Build params: {'input_dim': 768, 'output_dim': 4, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_bert.pb

In [289]:
pred_df = generic_model_bert.transform(testData_with_embeddings)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                  precision    recall  f1-score   support

Gastroenterology       1.00      0.92      0.96        25
       Neurology       0.74      0.91      0.82        22
      Orthopedic       0.90      0.80      0.85        35
         Urology       0.86      0.90      0.88        20

        accuracy                           0.87       102
       macro avg       0.88      0.88      0.88       102
    weighted avg       0.88      0.87      0.87       102

In [290]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericClassifier_bert"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### GenericLogRegClassifier

In [291]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_logreg_graph_builder = TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [292]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(30)\
    .setBatchSize(64)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [293]:
generic_model_bert = clf_Pipeline.fit(trainingData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 768, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb

In [294]:
pred_df = generic_model_bert.transform(testData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [295]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                  precision    recall  f1-score   support

Gastroenterology       0.96      0.92      0.94        25
       Neurology       0.81      0.77      0.79        22
      Orthopedic       0.88      0.86      0.87        35
         Urology       0.83      0.95      0.88        20

        accuracy                           0.87       102
       macro avg       0.87      0.87      0.87       102
    weighted avg       0.87      0.87      0.87       102

In [296]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericCLogReg_bert"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### GenericSVMClassifier

In [297]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_svm_graph_builder = TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [298]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(30)\
    .setBatchSize(64)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [299]:
generic_model_bert = clf_Pipeline.fit(trainingData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 768, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb

In [300]:
pred_df = generic_model_bert.transform(testData_with_embeddings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [301]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                  precision    recall  f1-score   support

Gastroenterology       0.96      0.92      0.94        25
       Neurology       0.75      0.95      0.84        22
      Orthopedic       0.93      0.80      0.86        35
         Urology       0.85      0.85      0.85        20

        accuracy                           0.87       102
       macro avg       0.87      0.88      0.87       102
    weighted avg       0.88      0.87      0.87       102

In [302]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericCSVM_bert"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## DocumentLogRegClassifier

In [303]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

logreg = DocumentLogRegClassifierApproach()\
    .setInputCols("token")\
    .setLabelCol("category")\
    .setOutputCol("prediction")\
    .setMaxIter(10)\
    .setTol(1e-6)

clf_Pipeline = Pipeline(stages=[document_assembler,
                                tokenizer,
                                logreg])

doclogreg_model = clf_Pipeline.fit(trainingData)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [304]:
pred_df = doclogreg_model.transform(testData)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                  precision    recall  f1-score   support

Gastroenterology       0.88      0.88      0.88        25
       Neurology       0.81      0.77      0.79        22
      Orthopedic       0.86      0.89      0.87        35
         Urology       0.90      0.90      0.90        20

        accuracy                           0.86       102
       macro avg       0.86      0.86      0.86       102
    weighted avg       0.86      0.86      0.86       102

In [305]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["DocumentLogRegClassifier"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Comparision of All Models' Result in Mtsamples Dataset

Here is the all results of classifier models. Macro f1, weighted f1 and accuracy scores are presented.

In [309]:
# results_df.to_html().replace('\n','')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [310]:
%%html
'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>macro-f1-score</th>      <th>weighted-f1-score</th>      <th>accuracy</th>    </tr>  </thead>  <tbody>    <tr>      <th>ClassifierDL_100d</th>      <td>0.576161</td>      <td>0.635125</td>      <td>0.705882</td>    </tr>    <tr>      <th>MultiClassifierDL_100d</th>      <td>0.885247</td>      <td>0.886721</td>      <td>0.887755</td>    </tr>    <tr>      <th>GenericClassifier_100d</th>      <td>0.889272</td>      <td>0.891150</td>      <td>0.892157</td>    </tr>    <tr>      <th>GenericLogReg_100d</th>      <td>0.847574</td>      <td>0.853001</td>      <td>0.852941</td>    </tr>    <tr>      <th>GenericSVM_100d</th>      <td>0.857103</td>      <td>0.861967</td>      <td>0.862745</td>    </tr>    <tr>      <th>ClassifierDL_200d</th>      <td>0.127737</td>      <td>0.175326</td>      <td>0.343137</td>    </tr>    <tr>      <th>MultiClassifierDL_200d</th>      <td>0.906453</td>      <td>0.907969</td>      <td>0.908163</td>    </tr>    <tr>      <th>GenericClassifier_200d</th>      <td>0.852994</td>      <td>0.859947</td>      <td>0.862745</td>    </tr>    <tr>      <th>GenericLogReg_200d</th>      <td>0.832973</td>      <td>0.841335</td>      <td>0.843137</td>    </tr>    <tr>      <th>GenericSVM_200d</th>      <td>0.838688</td>      <td>0.843115</td>      <td>0.843137</td>    </tr>    <tr>      <th>ClassifierDL_bert</th>      <td>0.883644</td>      <td>0.882072</td>      <td>0.882353</td>    </tr>    <tr>      <th>MultiClassifierDL_bert</th>      <td>0.884821</td>      <td>0.887095</td>      <td>0.886598</td>    </tr>    <tr>      <th>GenericClassifier_bert</th>      <td>0.875298</td>      <td>0.874269</td>      <td>0.872549</td>    </tr>    <tr>      <th>GenericCLogReg_bert</th>      <td>0.870690</td>      <td>0.872294</td>      <td>0.872549</td>    </tr>    <tr>      <th>GenericCSVM_bert</th>      <td>0.872578</td>      <td>0.873561</td>      <td>0.872549</td>    </tr>    <tr>      <th>DocumentLogRegClassifier</th>      <td>0.860984</td>      <td>0.862340</td>      <td>0.862745</td>    </tr>  </tbody></table>'

,macro-f1-score,weighted-f1-score,accuracy
ClassifierDL_100d,0.576161,0.635125,0.705882
MultiClassifierDL_100d,0.885247,0.886721,0.887755
GenericClassifier_100d,0.889272,0.891150,0.892157
GenericLogReg_100d,0.847574,0.853001,0.852941
GenericSVM_100d,0.857103,0.861967,0.862745
ClassifierDL_200d,0.127737,0.175326,0.343137
MultiClassifierDL_200d,0.906453,0.907969,0.908163
GenericClassifier_200d,0.852994,0.859947,0.862745
GenericLogReg_200d,0.832973,0.841335,0.843137
GenericSVM_200d,0.838688,0.843115,0.843137
